# Gather CESM2 SMYLE and DP runs 

In [1]:
import zarr
import numpy as np
import xarray as xr
from glob import glob

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## File info

In [3]:
# /glade/campaign/cesm/development/espwg/CESM2-DP/timeseries/b.e21.BSMYLE.f09_g17.2000-11.011/atm/proc/tseries/day_1
# 'TS': 'Surface temperature (radiative) [K]',
# 'TSMN': 'Surface Temperature Min [K]',
# 'TSMX': 'Surface temperature Max [K]',

varname_dict = {
    'SST': 'Surface Potential Temperature',
}

## Single year SYMLE + DP

In [4]:
year = 1958

In [5]:
base_case = 'b.e21.BSMYLE.f09_g17'
base_dir_DP = '/glade/campaign/cesm/development/espwg/CESM2-DP/timeseries/'
base_dir_SMYLE = '/glade/campaign/cesm/development/espwg/SMYLE/archive/'

**DP collect**

In [6]:
members = np.arange(11, 31, 1)
open_kwargs = dict(engine="netcdf4",) # chunks={"time": 6, "lev": 12, "lat": 192, "lon": 288}

ds_full_DP = []

for i_member, mem in enumerate(members):

    ds_collect = []
    
    str_member = f'{mem:03d}'
    fn_daily = base_dir_DP + f'{base_case}.{year}-11.{str_member}/' + '/ocn/proc/tseries/day_1/'
    
    for varname in list(varname_dict.keys()):
        fn_var = glob(fn_daily + f'*{varname}.*')[0]
        ds_collect.append(xr.open_dataset(fn_var, **open_kwargs)[[varname,]])
    
    ds_full_DP.append(xr.merge(ds_collect))

**SMYLE collect**

In [7]:
members = np.arange(11, 31, 1)
open_kwargs = dict(engine="netcdf4",) # chunks={"time": 6, "lev": 12, "lat": 192, "lon": 288}

ds_full_SMYLE = []

for i_member, mem in enumerate(members):

    ds_collect = []
    
    str_member = f'{mem:03d}'
    fn_daily = base_dir_SMYLE + f'{base_case}.{year}-11.{str_member}/' + '/ocn/proc/tseries/day_1/'
    
    for varname in list(varname_dict.keys()):
        fn_var = glob(fn_daily + f'*{varname}.*')[0]
        ds_collect.append(xr.open_dataset(fn_var, **open_kwargs)[[varname,]])
    
    ds_full_SMYLE.append(xr.merge(ds_collect))

**Merge**

In [8]:
ds_DP = xr.concat(ds_full_DP, dim=xr.DataArray(members, dims="member", name="member"),)
ds_DP = ds_DP.assign_coords(member=("member", members))
ds_DP = ds_DP.sel(time=slice(f'{year+2}-11-01T00', f'{year+10}-12-31T00'))

In [ ]:
ds_SMYLE = xr.concat(ds_full_SMYLE, dim=xr.DataArray(members, dims="member", name="member"),)
ds_SMYLE = ds_SMYLE.assign_coords(member=("member", members))
ds_SMYLE = ds_SMYLE.sel(time=slice(f'{year}-11-01T00', f'{year+2}-10-31T00'))

In [30]:
ds_year = xr.concat([ds_SMYLE, ds_DP], dim='time')
ds_year = ds_year.chunk({"member": 20, "time": 6, "nlat": 384, "nlon": 320})

In [ ]:
dict_encoding = {}
varnames = list(ds_year.keys())
varname_4D = []

chunk_size_3d = dict(chunks=(20, 6, 384, 320)) # member, time, lat, lon
chunk_size_4d = dict(chunks=(20, 6, 12, 384, 320)) # member, time, lev, lat, lon
compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)

for i_var, var in enumerate(varnames):
    if var in varname_4D:
        dict_encoding[var] = {'compressor': compress, **chunk_size_4d}
    else:
        dict_encoding[var] = {'compressor': compress, **chunk_size_3d}

In [4]:
save_name = f'/glade/derecho/scratch/ksha/EPRI_data/CESM_FULL/DP_{year}-11-01_daily_ensemble.zarr'
# # ds_year.to_zarr(save_name, mode='w', consolidated=True, compute=True, encoding=dict_encoding)

In [37]:
# ds_year = xr.open_zarr('/glade/derecho/scratch/ksha/EPRI_data/CESM_DP/DP_1993-11-01_daily_ensemble.zarr')
# ds_year.indexes['time'].is_monotonic_increasing

In [12]:
ds_full_DP[0]

<xarray.Dataset>
Dimensions:  (time: 2981, nlat: 384, nlon: 320)
Coordinates:
    ULONG    (nlat, nlon) float64 ...
    ULAT     (nlat, nlon) float64 ...
    TLONG    (nlat, nlon) float64 ...
    TLAT     (nlat, nlon) float64 ...
  * time     (time) object 1960-11-02 00:00:00 ... 1969-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables:
    SST      (time, nlat, nlon) float32 ...
Attributes:
    title:             b.e21.BSMYLE.f09_g17.1958-11.011
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  day_1
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    contents:          Diagnostic and Prognostic Variables
    source:            CCSM POP2, the CCSM Ocean Component
    revision:          $Id$
    calendar:          All years have exactly  365 days.
    start_time:        This dataset was created on 2024-02-27 at 14:57:47.4
    cell_methods:      cell_methods = time: mean ==> the variable values are ...